In [20]:
%load_ext autoreload
%autoreload 2
from src.models import HiddenMarkovModel, ViterbiAlgorithm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [263]:
prior_probabilities=np.array([0.67, 0.33])
transition_probabilities=np.array([[0.8, 0.2], 
                             [0.4, 0.6]]) 
emission_probabilities=np.array([[0.8, 0.2],
                           [0.4, 0.6]])
hidden_states = ['R01','R21']
observation_states = ['committed', 'ambivalent']

observation_states_dict = {observation_state: observation_state_index \
                                  for observation_state_index, observation_state in enumerate(list(observation_states))}

hidden_states_dict = {hidden_state_index: hidden_state \
                                   for hidden_state_index, hidden_state in enumerate(list(hidden_states))}

decode_observation_states=np.array(['committed', 'committed', 'ambivalent', 'ambivalent', 'ambivalent', 'committed'])
correct_hidden_states=np.array(['R01','R01', 'R21', 'R21', 'R21', 'R01'])

In [282]:
# Initialize path (i.e., np.arrays) to store the hidden sequence states returning the maximum probability
path = np.zeros((len(decode_observation_states), 
             len(hidden_states)))
path[0,:] = [hidden_state_index for hidden_state_index in range(len(hidden_states))]
best_path = np.zeros((len(decode_observation_states), 
             len(hidden_states)))        

# Compute initial delta:
# 1. Calculate the product of the prior and emission probabilities. This will be used to decode the first observation state.
# 2. Scale      

first_obs_idx = observation_states_dict[decode_observation_states[0]]
delta = prior_probabilities * emission_probabilities[:,first_obs_idx]
# traceback = np.array([0,1])

for node in range(1, len(decode_observation_states)):
    
    obs = decode_observation_states[node]
    obs_idx = observation_states_dict[obs]
    probs=[]
    
    for hidden_state_idx,hidden_state in enumerate(hidden_states):
        
        prob = delta[hidden_state_idx]*transition_probabilities[hidden_state_idx, :]*emission_probabilities[:, obs_idx]
        probs += [prob]

    prob_matrix = np.vstack([np.array(prob) for prob in probs])
    delta = np.max(prob_matrix, axis=0)    
    path[node] = np.argmax(prob_matrix, axis=0)
    
prev_idx = np.argmax(delta)
prev_val = hidden_states_dict[prev_idx]
best_path = [prev_val]
        
for i in range(1, path.shape[0])[::-1]:
    
    prev_idx = path[i, int(prev_idx)]
    best_path = [hidden_states_dict[prev_idx]] + best_path
    
print(best_path)

['R01', 'R01', 'R21', 'R21', 'R21', 'R01']


In [283]:
best_path == correct_hidden_states

array([ True,  True,  True,  True,  True,  True])